In [29]:
import Pkg; Pkg.add("CSV")
using CSV
import Pkg; Pkg.add("Base")
using Base.Iterators: flatten

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


Pkg.Types.PkgError: The following package names could not be resolved:
 * Base (not found in project, manifest or registry)
[36m   Suggestions:[39m [0m[1mB[22m[0m[1ma[22m[0m[1ms[22m[0m[1me[22m58 [0m[1mB[22m[0m[1ma[22m[0m[1ms[22m[0m[1me[22m64 [0m[1mB[22m[0m[1ma[22m[0m[1ms[22m[0m[1me[22mlet [0m[1mB[22m[0m[1ma[22m[0m[1ms[22m[0m[1me[22msAndSamples Q[0m[1mB[22m[0m[1ma[22m[0m[1ms[22m[0m[1me[22m ML[0m[1mB[22m[0m[1ma[22m[0m[1ms[22m[0m[1me[22m XC[0m[1mB[22m[0m[1ma[22m[0m[1ms[22m[0m[1me[22md

In [30]:
Article{T} = Pair{T, T}
art_map(f, a) = f(a.first) => f(a.second)

art_map (generic function with 1 method)

In [31]:
function read_csvs(files::Vector{String})
    data = Set{Article{String}}()
    for row in flatten(map(CSV.File, files))
        art = (row.clean_title=> row.clean_text)
        any(ismissing, art) ? continue : push!(data, art)
    end
    collect(data)
end

read_csvs (generic function with 1 method)

In [56]:
data_dir = "data"
files = joinpath.(data_dir, readdir(data_dir))
data = read_csvs(files)
unique!(a->a.first, data)
unique!(a->a.second, data)
data |> rand

"навык поиска работы" => "есть много причин почему классные разработчики часто долго не могут найти работу а компании  классных разработчиков я хочу поговорить об одной из наиболее распространённыхэта причина  абсолютно неправильный деструктивный подход к проведению технических собеседований это похоже на инфекционное заболевание которым болеют интервьюеры и заражают тех кого они собеседуют некоторые из тех кого они собеседуют потом тоже сами становятся интервьюерами и заражают тех кого они собеседуют и так далеепроблема в том что it-индустрия предполагает что любой опытный разработчик умеет собеседовать других разработчиков откуда такое убеждение это же вывод не основанный ни на чём он этому обучался он что-то специально об этом читал он готовился не во всех случаях разумеется но чаще чем хотелось бы ответ на все три вопроса  нет потому что и так прокатывает тебе говорят  ищем сотрудника под новую вакансию завтра у тебя собеседование а у тебя кроме этого собеседования ещё задач стольк

In [33]:
function tokenize(s::String, pattern = r"\w+(-\w+)?")
    lowered = lowercase(s)
    map(eachmatch(pattern, lowered)) do token
        token.match
    end
end

tokenize(a::Article{String}) = art_map(tokenize, a)

tokenize (generic function with 3 methods)

In [57]:
data = tokenize.(data)
data |> rand

SubString{String}["монорепозитории", "пожалуйста", "не", "надо", "часть", "2"] => SubString{String}["всем", "приветитак", "новая", "порция", "обещанного", "холивара", "про", "монорепозитории", "в", "первой"  …  "компаний", "которые", "умеют", "в", "монорепозиторий", "остальным", "даже", "не", "стоит", "пытаться"]

In [35]:
counts = Dict{String, Int}()
for art in data, token in union(art.first, art.second)
    counts[token] = get(counts, token, 0) + 1
end
filter!(kv->kv.second > 20, counts)

Dict{String, Int64} with 1809 entries:
  "нужный"        => 21
  "комментарии"   => 22
  "той"           => 56
  "похоже"        => 29
  "способом"      => 21
  "данной"        => 50
  "этими"         => 33
  "32"            => 29
  "думаю"         => 35
  "которые"       => 288
  "понять"        => 79
  "представить"   => 33
  "получите"      => 21
  "80"            => 27
  "нечто"         => 30
  "автоматически" => 43
  "ранее"         => 61
  "выхода"        => 29
  "20"            => 96
  ⋮               => ⋮

In [36]:
tokens = collect(keys(counts))
pushfirst!(tokens, "<UNK>")
lookup = Dict(reverse.(enumerate(tokens)))
Tokens = Vector{SubString{String}}
encode(lookup, tokens::Tokens) = get.(Ref(lookup), tokens, 1)
encode(lookup, art::Article{Tokens}) = art_map(x->encode(lookup, x), art) 

encode (generic function with 2 methods)

In [59]:
length(data)

388

In [58]:
data = encode.(Ref(lookup), data)
filter!(art->any(!isone, art.first), data)
filter!(art->any(!isone, art.second), data)
#filter!(art->10 < length(art.second) < 100, data)
#filter!(art->5 < length(art.first) < 20, data)

#filter!(data) do art
    #i = length(intersect(art.first, art.second))
    #u = length(union(art.first, art.second))
    #i / u < 0.5
#end

data |> rand

[269, 82, 1, 391, 1, 668, 1, 1, 1] => [1, 294, 1, 1, 926, 1, 1, 1, 307, 1393  …  422, 1, 1, 1, 943, 82, 1, 1, 1, 1]

In [39]:
import Pkg; Pkg.add("BSON")

   Resolving package versions...


   Installed BSON ─ v0.3.5
    Updating `~/.julia/environments/v1.8/Project.toml`
  [fbb218c0] + BSON v0.3.5
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [fbb218c0] + BSON v0.3.5


Precompiling project...


  ✓ BSON
  1 dependency successfully precompiled in 1 seconds. 21 already precompiled.


In [60]:
using BSON: @save
n_test = 150
train_data = collect.(data[(n_test+1):end])
test_data = collect.(data[1:n_test])
@save "data.bson" tokens train_data test_data